In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install mljar-supervised

In [ ]:
#Library Imports
import random
import pandas as pd
import numpy as np
import os
import gc

import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

from supervised.automl import AutoML

import warnings
warnings.filterwarnings(action='ignore')

pd.options.display.max_rows = 1000

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Power/data/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Power/data/test.csv')
buildings = pd.read_csv('/content/drive/MyDrive/Power/data/building_info.csv')
submission=pd.read_csv('/content/drive/MyDrive/Power/data/sample_submission.csv')

## Fixed Random-Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## 전처리

In [ ]:
train = train.drop(columns=['num_date_time'])
test = test.drop(columns=['num_date_time'])

In [ ]:
train.columns = ['num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'sun', 'solar', 'power']
test.columns = ['num', 'date_time', 'temp', 'prec', 'wind', 'hum']

In [ ]:
train = train.drop(columns=['prec', 'wind', 'hum', 'sun', 'solar'])
test = test.drop(columns=['prec', 'wind', 'hum'])

### datetime 변환

In [ ]:
train_date = pd.to_datetime(train.date_time)
test_date = pd.to_datetime(test.date_time)

train['date_time'] = pd.to_datetime(train.date_time)
train['hour'] = train_date.dt.hour
train['day'] = train_date.dt.weekday
train['month'] = train_date.dt.month
train['week'] = train_date.dt.weekofyear

test['date_time'] = pd.to_datetime(test.date_time)
test['hour'] = test_date.dt.hour
test['day'] = test_date.dt.weekday
test['month'] = test_date.dt.month
test['week'] = test_date.dt.weekofyear

In [ ]:
train['week'] = (pd.to_datetime(train['date_time']) - pd.to_datetime('2022-06-02')).dt.days // 7
test['week'] = (pd.to_datetime(test['date_time']) - pd.to_datetime('2022-06-02')).dt.days // 7

In [ ]:
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)

### 결측치 처리

In [ ]:
train = train[train['date_time'] >= '2022-06-30 00:00:00']

In [ ]:
num_01_index = train[train['num']==1][train[train['num']==1]['date_time'] < '2022-07-11 00:00:00'].index
num_03_index = train[train['num']==3][train[train['num']==3]['date_time'] < '2022-07-22 00:00:00'].index
num_07_index = train[train['num']==7][train[train['num']==7]['date_time'] < '2022-07-06 00:00:00'].index
num_13_index = train[train['num']==13][train[train['num']==13]['date_time'] < '2022-07-25 00:00:00'].index
num_14_index = train[train['num']==14][train[train['num']==14]['date_time'] < '2022-07-03 00:00:00'].index
num_17_index = train[train['num']==17][train[train['num']==17]['date_time'] >= '2022-07-23 00:00:00'][train[train['num']==17][train[train['num']==17]['date_time'] >= '2022-07-23 00:00:00']['date_time'] <= '2022-07-23 23:00:00'].index
num_21_index = train[train['num']==21][train[train['num']==21]['date_time'] < '2022-07-04 00:00:00'].index
num_22_index = train[train['num']==22][train[train['num']==22]['date_time'] < '2022-07-11 00:00:00'].index
num_36_index = train[train['num']==36][train[train['num']==36]['date_time'] >= '2022-07-31 00:00:00'][train[train['num']==36][train[train['num']==36]['date_time'] >= '2022-07-31 00:00:00']['date_time'] <= '2022-07-31 23:00:00'].index
num_54_index = train[train['num']==54][train[train['num']==54]['date_time'] >= '2022-08-16 00:00:00'][train[train['num']==54][train[train['num']==54]['date_time'] >= '2022-08-16 00:00:00']['date_time'] <= '2022-08-17 23:00:00'].index
num_58_index = train[train['num']==58][train[train['num']==58]['date_time'] < '2022-07-06 00:00:00'].index
num_70_index  = train[train['num']==70][train[train['num']==70]['date_time'] >= '2022-07-29 06:00:00'][train[train['num']==70][train[train['num']==70]['date_time'] >= '2022-07-29 06:00:00']['date_time'] <= '2022-08-08 05:00:00'].index
num_71_index = train[train['num']==71][train[train['num']==71]['date_time'] >= '2022-07-30 00:00:00'][train[train['num']==71][train[train['num']==71]['date_time'] >= '2022-07-30 00:00:00']['date_time'] <= '2022-07-30 23:00:00'].index
num_86_index = train[train['num']==86][train[train['num']==86]['date_time'] >= '2022-07-30 00:00:00'][train[train['num']==86][train[train['num']==86]['date_time'] >= '2022-07-30 00:00:00']['date_time'] <= '2022-07-30 23:00:00'].index
num_89_index = train[train['num']==89][train[train['num']==89]['date_time'] >= '2022-07-09 00:00:00'][train[train['num']==89][train[train['num']==89]['date_time'] >= '2022-07-09 00:00:00']['date_time'] <= '2022-07-29 23:00:00'].index
num_91_index_01 = train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-10 00:00:00'][train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-10 00:00:00']['date_time'] <= '2022-07-10 23:00:00'].index
num_91_index_02 = train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-11 00:00:00'][train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-11 00:00:00']['date_time'] <= '2022-07-11 23:00:00'].index
num_91_index_03 = train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-24 00:00:00'][train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-24 00:00:00']['date_time'] <= '2022-07-24 23:00:00'].index
num_91_index_04 = train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-30 00:00:00'][train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-07-30 00:00:00']['date_time'] <= '2022-07-30 23:00:00'].index
num_91_index_05 = train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-08-14 00:00:00'][train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-08-14 00:00:00']['date_time'] <= '2022-08-14 23:00:00'].index
num_91_index_06 = train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-08-22 00:00:00'][train[train['num']==91][train[train['num']==91]['date_time'] >= '2022-08-22 00:00:00']['date_time'] <= '2022-08-22 23:00:00'].index
num_92_index = train[train['num']==92][train[train['num']==92]['date_time'] >= '2022-07-30 00:00:00'][train[train['num']==92][train[train['num']==92]['date_time'] >= '2022-07-30 00:00:00']['date_time'] <= '2022-07-30 23:00:00'].index
num_95_index = train[train['num']==95][train[train['num']==95]['date_time'] >= '2022-07-27 00:00:00'][train[train['num']==95][train[train['num']==95]['date_time'] >= '2022-07-27 00:00:00']['date_time'] <= '2022-07-29 23:00:00'].index

In [ ]:
train = train.drop(num_01_index) #1번 건물 7월 11일 이후 데이터 사용
train = train.drop(num_03_index) #3번 건물 7월 22일 이후 데이터 사용, 3번 건물 이상치 데이터 더 있음
train['power'][7438] = 684.81 #4번 건물 이상치 처리
train['power'][7439] = 632.19 #4번 건물 이상치 처리
train = train.drop(num_07_index) #7번 건물 7월 06일 이후 데이터 사용
train['power'][13818] = 1122.30 #7번 건물 이상치 처리
train['power'][13819] = 976.68 #7번 건물 이상치 처리
train['power'][13820] = 831.06 #7번 건물 이상치 처리
train['power'][21700] = 1648.08 #11번 건물 이상치 처리
train['power'][21701] = 1677.60 #11번 건물 이상치 처리
train['power'][21893] = 1638.12 #11번 건물 이상치 처리
train['power'][22057] = 1729.17 #11번 건물 이상치 처리
train['power'][22058] = 1713.58 #11번 건물 이상치 처리
train['power'][22059] = 1697.99 #11번 건물 이상치 처리
train['power'][22065] = 2164.68 #11번 건물 이상치 처리
train['power'][22242] = 1949.68 #11번 건물 이상치 처리
train['power'][22243] = 1948.64 #11번 건물 이상치 처리
train = train.drop(num_13_index) #13번 건물 7월 25일 이후 데이터 사용
train = train.drop(num_14_index) #14번 건물 7월 03일 이후 데이터 사용
train = train.drop(num_17_index) #17번 건물 7월 23일 데이터 제거
train = train.drop(num_21_index) #14번 건물 7월 04일 이후 데이터 사용
train = train.drop(num_22_index) #14번 건물 7월 11일 이후 데이터 사용
train['power'][52089] = 2674.08 #26번 건물 이상치 처리
train['power'][52090] = 2724.08 #26번 건물 이상치 처리
train['power'][56450] = 1108.26 #28번 건물 이상치 처리
train['power'][61955] = 1038.96 #31번 건물 이상치 처리
train['power'][68973] = 3471.84 #34번 건물 이상치 처리
train['power'][71013] = 2200.80 #35번 건물 이상치 처리
train['power'][71014] = 2216.40 #35번 건물 이상치 처리
train = train.drop(num_36_index) #36번 건물 7월 31일 데이터 제거
train['power'][104860] = 1784.70 #52번 건물 이상치 처리
train['power'][105456] = 1642.50 #52번 건물 이상치 처리
train = train.drop(num_54_index) #54번 건물 8월 16,17일 데이터 제거
train = train.drop(num_58_index) #58번 건물 7월 06일 이후 데이터 사용
train['power'][123132] = 2557.26 #61번 건물 이상치 처리
train = train.drop(num_70_index) #70번 건물 7월 29일~8월 8일 데이터 제거
train = train.drop(num_71_index) #71번 건물 7월 30일 데이터 제거
train['power'][145786] = 1178.22
train['power'][145787] = 1182.48
train['power'][145954] = 1047.24
train['power'][146289] = 1099.53
train['power'][150883] = 2631.6
train['power'][150884] = 2670.48
train['power'][150885] = 2709.36
train['power'][150886] = 2748.24
train = train.drop(152393)
train = train.drop(num_86_index)
train = train.drop(num_89_index)
train = train.drop(num_91_index_01)
train = train.drop(num_91_index_02)
train = train.drop(num_91_index_03)
train = train.drop(num_91_index_04)
train = train.drop(num_91_index_05)
train = train.drop(num_91_index_06)
train = train.drop(num_92_index)
train = train.drop(num_95_index)

In [ ]:
holiday_num = [15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 82, 83, 84]
No_holiday_num = [num for num in range(1, 101) if num not in holiday_num]

In [ ]:
train = train.drop(train[train['num'].isin(holiday_num) & (train['date_time'] >= '2022-08-15 00:00:00') & (train['date_time'] <= '2022-08-15 23:00:00')].index).reset_index(drop=True)

In [ ]:
dates_to_remove = {
    7: [('2022-07-17 00:00:00', '2022-07-17 23:00:00'), ('2022-08-21 00:00:00', '2022-08-21 23:00:00')],
    11: [('2022-07-10 00:00:00', '2022-07-10 23:00:00'), ('2022-07-24 00:00:00', '2022-07-24 23:00:00'), ('2022-07-30 00:00:00', '2022-07-30 23:00:00')],
    12: [('2022-07-02 00:00:00', '2022-07-02 23:00:00'), ('2022-07-03 00:00:00', '2022-07-03 23:00:00'), ('2022-07-17 00:00:00', '2022-07-17 23:00:00'), ('2022-08-07 00:00:00', '2022-08-07 23:00:00'), ('2022-08-21 00:00:00', '2022-08-21 23:00:00')],
    37: [('2022-07-11 00:00:00', '2022-07-11 23:00:00'), ('2022-08-08 00:00:00', '2022-08-08 23:00:00')],
    38: [('2022-07-25 00:00:00', '2022-07-25 23:00:00'), ('2022-08-01 00:00:00', '2022-08-01 23:00:00')],
    39: [('2022-07-18 00:00:00', '2022-07-18 23:00:00'), ('2022-08-08 00:00:00', '2022-08-08 23:00:00')],
    40: [('2022-07-18 00:00:00', '2022-07-18 23:00:00'), ('2022-08-08 00:00:00', '2022-08-08 23:00:00')],
    41: [('2022-07-25 00:00:00', '2022-07-25 23:00:00'), ('2022-08-08 00:00:00', '2022-08-08 23:00:00')],
    42: [('2022-07-11 00:00:00', '2022-07-11 23:00:00'), ('2022-08-22 00:00:00', '2022-08-22 23:00:00')],
}

for num, date_ranges in dates_to_remove.items():
    for start_date, end_date in date_ranges:
        condition = (train['num'] == num) & (train['date_time'] >= start_date) & (train['date_time'] <= end_date)
        train = train.drop(train[condition].index).reset_index(drop=True)

## Submission

In [ ]:
total_rows = len(submission)
num_submissions = total_rows // 168
rows_per_submission = 168
submission_list = []

for i in range(num_submissions):
    start_idx = i * rows_per_submission
    end_idx = start_idx + rows_per_submission
    submission_part = submission.iloc[start_idx:end_idx]
    submission_list.append(submission_part)

In [ ]:
day_off = [6, 9, 86, 87, 88, 89, 90, 92]
day_on = [i for i in range(1, 101) if i not in day_off]

In [ ]:
for i in day_on[10:20]:
    print(i,'번째 건물')
    X_train = train[train['num']==i].reset_index(drop=True)
    X_test = test[test['num']==i].reset_index(drop=True)

    scaler = MinMaxScaler()
    X_train['temp'] = scaler.fit_transform(X_train[['temp']])
    X_test['temp'] = scaler.transform(X_test[['temp']])

    y_train = X_train['power']
    X_train = X_train.drop(columns=['num', 'date_time', 'power'])
    X_test = X_test.drop(columns=['num', 'date_time'])

    automl = AutoML(mode="Compete", eval_metric='mae', algorithms=['LightGBM', 'CatBoost', 'Xgboost'], random_state = 42)
    automl.fit(X_train, y_train)
    preds = automl.predict(X_test)
    submission_list[i-1]['answer'] = preds

    submission_list[i-1].to_csv(f'/content/drive/MyDrive/Power/Num_{i}.csv', index=False)

    gc.collect()

13 번째 건물
AutoML directory: AutoML_1
The task is regression with evaluation metric mae
AutoML will use algorithms: ['LightGBM', 'CatBoost', 'Xgboost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 140.674385 trained in 0.56 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_LightGBM mae 88.686835 trained in 8.7 seconds
2_Default_Xgboost mae 93.229886 trained in 4.71 seconds
3_Default_CatBoost mae 82.725527 trained in 12.28 seconds
* Step not_so_random will